In [1]:
from pathlib import Path ; from shutil import copytree  ; from tempfile import TemporaryDirectory ; import pandas as pd
import numpy as np ;  import flopy ;  import os ; import pathlib ; import numpy as np; import matplotlib.pyplot as plt

In [2]:
name = "tutorial03_mf6_data"                  # sim_name
exe_name = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6'
current_directory = os.getcwd();  print(current_directory)

E:\15_REPOS\00_BETAMI\01_floPy\002_mf6


In [3]:
path = pathlib.Path('E:\\15_REPOS\\00_BETAMI\\01_floPy\\002_mf6')
path.mkdir(parents=True, exist_ok=True); os.chdir(path); current_directory = os.getcwd()
directory_name = 'A2_v01'    ; workspace = os.path.join('.', directory_name)                         # sim_ws
if os.path.isdir(workspace) is False:
    os.mkdir(workspace) 
output_folder = Path('A2_v01') ; output_folder.mkdir(exist_ok=True)     

In [4]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=workspace, exe_name="mf6", version="mf6")
tdis_rc = [(1.0, 1, 1.0), (10.0, 5, 1.0), (10.0, 5, 1.0), (10.0, 1, 1.0)]

tdis_package   = flopy.mf6.modflow.mftdis.ModflowTdis(sim, time_units="DAYS", nper=4, perioddata=tdis_rc)
model_nam_file = f"{name}.nam"
gwf = flopy.mf6.ModflowGwf                     (sim, modelname=name, model_nam_file = model_nam_file)
ims = flopy.mf6.modflow.mfims.ModflowIms       (sim, pname="ims", complexity="SIMPLE")      ; bot= np.linspace(-3 ,-50/3 ,3)  ; delrow = delcol = 4.0
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis (gwf, pname="dis",nogrb=True,nlay=3,nrow=101,ncol=101,delr=delrow,delc=delcol  , top=0.0, botm=bot)
ic_package = flopy.mf6.modflow.mfgwfic.ModflowGwfic    (gwf, strt=50.0)
npf_package = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf (gwf,save_flows=True,icelltype=[1, 0, 0], k=[5.0, 0.1, 4.0], k33=[0.5, 0.005, 0.1])

ghb_spd_ts = {}
ghb_period = []
for layer, cond in zip(range(1, 3), [15.0, 1500.0]):
    for row in range(0, 15):
        ghb_period.append(((layer, row, 9), "tides", cond, "Estuary-L2"))
ghb_spd_ts[0] = ghb_period

ts_data = []
for n in range(0, 365):
    time = float(n / 11.73)  ; val = float(n / 60.0)  ; ts_data.append((time, val))
ts_dict = {"filename": "tides.ts","time_series_Namerec": "tide","ts": ts_data,"interpolation_methodrecord": "linearend","sfacrecord": 1.1}

ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(gwf,print_input=True,print_flows=True,save_flows=True,
                                               boundnames=True,pname="ghb",maxbound=30, stress_period_data = ghb_spd_ts)
#ghb.ts.time_series_namerecord = "tides"

print("{} is using {} interpolation".format(ghb.ts[0].filename, ghb.ts[0].interpolation_methodrecord.get_data()[0][0]))
print("{} is using {} interpolation".format(ghb.ts[1].filename, ghb.ts[1].interpolation_methodrecord.get_data()[0][0]))
print("{} is using {} interpolation".format(ghb.ts[2].filename, ghb.ts[2].interpolation_methodrecord.get_data()[0][0]))

sim.write_simulation()

C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\gross.DESKTOP-AIRQM2M\anaconda3\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value in

ValueError: Package index 0 does not exist.